In [1]:
import sys
import os
sys.path.append("/home/alexserra98/helm-suite")
sys.path.append("/home/alexserra98/helm-suite/inference_id")

In [2]:
from metrics.metrics import *
import pickle
import copy

/home/alexserra98/miniconda3/envs/crfm-helm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open('letter_overlap.pkl', 'rb') as f:
    lo = pickle.load(f)

In [5]:
lo

{'last': [array([[1.  , 2.  , 1.  , 2.  ],
         [1.86, 1.29, 1.14, 1.71],
         [1.29, 1.71, 1.14, 1.86],
         [1.14, 1.57, 1.14, 2.14]]),
  array([[0.86, 1.71, 1.43, 2.  ],
         [1.14, 1.  , 1.71, 2.14],
         [0.86, 1.86, 1.  , 2.29],
         [0.57, 2.  , 2.  , 1.43]]),
  array([[1.  , 1.57, 1.29, 2.14],
         [1.29, 1.14, 1.43, 2.14],
         [1.14, 1.43, 1.14, 2.29],
         [1.29, 1.71, 1.43, 1.57]]),
  array([[1.  , 1.29, 1.71, 2.  ],
         [1.43, 0.86, 1.43, 2.29],
         [1.29, 1.57, 1.29, 1.86],
         [1.29, 1.71, 1.43, 1.57]]),
  array([[1.14, 1.29, 1.71, 1.86],
         [1.57, 1.  , 1.29, 2.14],
         [1.57, 1.57, 1.14, 1.71],
         [1.57, 1.57, 1.43, 1.43]]),
  array([[1.29, 1.29, 1.57, 1.86],
         [1.43, 1.  , 1.29, 2.29],
         [1.86, 1.57, 0.71, 1.86],
         [1.29, 1.86, 1.43, 1.43]]),
  array([[1.29, 2.  , 0.86, 1.86],
         [1.43, 1.14, 1.14, 2.29],
         [1.86, 1.43, 1.29, 1.43],
         [1.29, 1.86, 0.86, 2.  ]])

In [7]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(z=lo["last"][0], colorscale='hot'))
fig.show()

In [6]:
with open('tests/assets/requests_results.pkl', 'rb') as f:
    request_result = pickle.load(f)

In [12]:
request_result.append(copy.deepcopy(request_result[-1]))

In [13]:
for i,letter in zip(request_result, ["A","A","B","B","A","A","B","B","C","C","D","D","B","B","D","D"]):
    #print(i.preds["only_ref_pred"]["letter"],letter)
    i.preds["only_ref_pred"]["letter"] = letter
    print(i.preds["only_ref_pred"]["letter"],letter)


A A
A A
B B
B B
A A
A A
B B
B B
C C
C C
D D
D D
B B
B B


In [29]:
request_result[-7].preds["only_ref_pred"]["letter"] = "C"

In [30]:
for i,letter in zip(request_result, ["A","A","A","A","B","B","B","B","C","C","C","C","D","D","D","D"]):
    print(i.preds["only_ref_pred"]["letter"],letter)

A A
A A
C A
A A
B B
B B
D B
D B
C C
C C
C C
A C
B D
B D
D D
D D


In [46]:
request_result[5]


RequestResult(loss=tensor(10.5277), logits=tensor([[-136.5588, -136.2273, -138.3011,  ..., -142.5228, -143.0245,
         -134.2092]]), hidden_states={'last': array([[ 0.04, -0.19,  0.05, ...,  0.03, -0.08, -0.03],
       [-0.5 , -0.22,  0.35, ...,  0.41, -0.05,  0.43],
       [-0.5 , -0.35,  0.24, ...,  0.52,  0.25,  0.28],
       ...,
       [ 3.48, -0.88, -2.22, ...,  2.64,  5.41,  4.24],
       [ 4.3 , -0.44, -2.49, ...,  3.93,  6.28,  8.61],
       [ 0.44, -0.08, -0.12, ...,  0.34,  0.31,  0.74]], dtype=float32), 'sum': array([[ 2.79e-02,  3.37e-03,  1.28e-01, ..., -5.22e-05,  1.14e-02,
         1.59e-02],
       [ 2.84e-01,  2.63e-01,  2.97e-01, ...,  2.46e-01, -4.68e-02,
        -8.76e-02],
       [ 3.21e-01,  4.21e-01,  1.30e-01, ...,  3.67e-01,  3.46e-01,
        -2.18e-01],
       ...,
       [ 1.11e+00,  2.25e+00, -1.01e+00, ...,  4.44e-01, -1.33e-01,
         2.99e-01],
       [ 1.11e+00,  2.09e+00, -1.34e+00, ...,  2.94e-01, -4.60e-02,
         8.25e-01],
       [ 2.65e-01

In [32]:
with open('tests/assets/requests_results.pkl', 'wb') as f:
    pickle.dump(request_result, f)